# BQPlot

* Interactive plotting library developed by Bloomberg Quants, especially **Srinivas Sunkara, Chakri Cherukuri, Sylvain Corlay, Dhruv Madeka, and Romain Menegaux**
* Based on IPython widgets
* Open source

# https://github.com/bloomberg/bqplot

# BQPlot Example

In [ ]:
from bqplot import pyplot as plt
from numpy import random, cumsum, linspace
random.seed(0)
size = 100
y_data = cumsum(random.randn(size) * 100.0)
y_data_2 = cumsum(random.randn(size))
y_data_3 = cumsum(random.randn(size) * 100.)


In [ ]:
plt.figure(1)
n = 100
x = linspace(0.0, 10.0, n)
plt.plot(x, y_data, axes_options={'y': {'grid_lines': 'dashed'}})
plt.show()

In [ ]:
from IPython.display import clear_output
def print_value(name, value):
    clear_output()
    print(value)
plt.brush_int_selector(print_value)

# Scatter Plot

In [ ]:
import pandas as pd
import numpy as np
from bqplot import *
from bqplot.interacts import *
from traitlets import link

from IPython.display import display
from ipywidgets import Label, ToggleButtons, VBox

date_fmt = '%m-%d-%Y'
symbol = 'Security 1'
from datetime import datetime as py_dtime
price_data = pd.DataFrame(np.cumsum(np.random.randn(150, 2).dot([[0.5, 0.4], [0.4, 1.0]]), axis=0) + 100,
                          columns=['Security 1', 'Security 2'], index=pd.date_range(start='01-01-2007', periods=150))

dates_actual = price_data.index.values
prices = price_data[symbol].values
symbol2 = 'Security 2'
date_fmt = '%m-%d-%Y'

sec2_data = price_data[symbol2].values
dates = price_data.index.values

from bqplot.market_map import MarketMap

In [ ]:
sc_x = LinearScale()
sc_y = LinearScale()
sc_e = LinearScale()
sc_c = ColorScale(scheme='Reds')

ax_x = Axis(scale=sc_x)
ax_y = Axis(scale=sc_y, orientation='vertical')
ax_c = ColorAxis(scale=sc_c, orientation='vertical', side='right')

x1 = np.linspace(-1, 1, 20)
y1 = np.linspace(-1, 1, 20)
x, y = np.meshgrid(x1,y1)
x, y = x.flatten(), y.flatten()
rot = x**2 + y**2
color=x-y
scatter = Scatter(scales={'x': sc_x, 'y': sc_y, 'color': sc_c, 'rotation': sc_e}, 
                  x=x, y=y, rotation=rot, color=color,
                  stroke="black", animate_dur=1000, default_size=200, 
                  marker='arrow', default_skew=0.5,)
Figure(marks=[scatter], axes=[ax_x, ax_y, ax_c])

In [ ]:
scatter.rotation = 1.0 / (x ** 2 + y ** 2 + 1)
scatter.color = x+y

# Scatter Interaction

In [ ]:
size = 100
np.random.seed(0)
x_data = range(size)
y_data = np.cumsum(np.random.randn(size) * 100.0)

## Enabling moving of points in scatter. Try to click and drag any of the points in the scatter and 
## notice the line representing the mean of the data update
sc_x = LinearScale()
sc_y = LinearScale()

scat = Scatter(x=x_data[:10], y=y_data[:10], scales={'x': sc_x, 'y': sc_y}, default_colors=['orange'],
               enable_move=True)
lin = Lines(scales={'x': sc_x, 'y': sc_y}, line_style='dotted', colors=['orange'])
m = Label(value='Mean is %s'%np.mean(scat.y))

def update_line(change):
    with lin.hold_sync():
        lin.x = [np.min(scat.x), np.max(scat.x)]
        lin.y = [np.mean(scat.y), np.mean(scat.y)]
        m.value='Mean is %s'%np.mean(scat.y)
        

update_line(None)
# update line on change of x or y of scatter
scat.observe(update_line, names='x')
scat.observe(update_line, names='y')

ax_x = Axis(scale=sc_x)
ax_y = Axis(scale=sc_y, tick_format='0.2f', orientation='vertical')

fig = Figure(marks=[scat, lin], axes=[ax_x, ax_y])

## In this case on drag, the line updates as you move the points.
with scat.hold_sync():
    scat.enable_move = True
    scat.update_on_move = True
    scat.enable_add = False

    
display(m, fig)

# More Interactions

In [ ]:
dt_x = DateScale(date_format=date_fmt, min=py_dtime(2007, 1, 1))
lc1_x = LinearScale()
lc2_y = LinearScale()

lc2 = Lines(x=np.linspace(0.0, 10.0, len(prices)), y=prices * 0.25,
            scales={'x': lc1_x, 'y': lc2_y}, 
            display_legend=True,
            labels=['Security 1'])

lc3 = Lines(x=dates_actual, y=sec2_data,
            scales={'x': dt_x, 'y': lc2_y},
            colors=['red'], 
            display_legend=True, 
            labels=['Security 2'])

lc4 = Lines(x=np.linspace(0.0, 10.0, len(prices)), y=sec2_data * 0.75,
            scales={'x': LinearScale(min=5, max=10), 'y': lc2_y},
            colors=['green'], display_legend=True, 
            labels=['Security 2 squared'])

x_ax1 = Axis(label='Date', scale=dt_x)
x_ax2 = Axis(label='Time', scale=lc1_x, side='top')
x_ay2 = Axis(label=(symbol + ' Price'), scale=lc2_y, orientation='vertical', tick_format='0.2f', label_offset = '50px')


fig = Figure(marks=[lc2, lc3, lc4], axes=[x_ax1, x_ax2, x_ay2])

In [ ]:
## declaring the interactions
multi_sel = MultiSelector(scale=dt_x, marks=[lc2, lc3])
br_intsel = BrushIntervalSelector(scale=lc1_x, marks=[lc2, lc3])
index_sel = IndexSelector(scale=dt_x, marks=[lc2, lc3])
int_sel = FastIntervalSelector(scale=dt_x, marks=[lc3, lc2])

hd = HandDraw(lines=lc2)
hd2 = HandDraw(lines=lc3)
pz = PanZoom(scales={'x': [dt_x], 'y': [lc2_y]})

deb = Label()
deb.value = "hello"

In [ ]:
## Call back handler for the interactions
def test_callback(change):
    value = change['new']
    deb.value = str(value)
    
multi_sel.observe(test_callback, names='selected')
br_intsel.observe(test_callback, names='selected')
index_sel.observe(test_callback, names='selected')
int_sel.observe(test_callback, names='selected')

In [ ]:
from collections import OrderedDict
selection_interacts = ToggleButtons(options=OrderedDict([('HandDraw1', hd), ('HandDraw2', hd2), ('PanZoom', pz), 
                                                       ('FastIntervalSelector', int_sel), ('IndexSelector', index_sel),
                                                       ('BrushIntervalSelector', br_intsel),
                                                       ('None', None)]))
display(deb)
display(VBox([fig, selection_interacts], align_self="stretch"))

link((selection_interacts, 'value'), (fig, 'interaction'));

# Country GDP

In [ ]:
data = pd.read_csv('data_files/country_codes.csv', index_col=[0])
country_codes = data.index.values
country_names = data['Name']

In [ ]:
gdp_data = pd.read_csv('data_files/gdp_per_capita.csv', index_col=[0], parse_dates=True)
gdp_data.fillna(method='backfill', inplace=True)
gdp_data.fillna(method='ffill', inplace=True)

In [ ]:
col = ColorScale(scheme='Greens')
continents = data['Continent'].values
ax_c = ColorAxis(scale=col, label='GDP per Capita', visible=False)

data['GDP'] = gdp_data.ix[-1]

market_map = MarketMap(names=country_codes, groups=continents,       # Basic data which needs to set for each map
                       cols=25, row_groups=3,                        # Properties for the visualization
                       ref_data=data,                                # Data frame used for different properties of the map
                       tooltip_fields=['Name', 'Continent', 'GDP'],  # Columns from data frame to be displayed as tooltip
                       tooltip_formats=['', '', '.1f'],
                       scales={'color': col}, axes=[ax_c])           # Axis and scale for color data

In [ ]:
deb_output = Label()
def selected_index_changed(change):
    value = change['new']
    deb_output.value = str(value)
        
market_map.observe(selected_index_changed, names='selected')
display(deb_output)
display(market_map)

In [ ]:
# Attribute to show the names of the groups, in this case the continents
market_map.show_groups = True

In [ ]:
market_map.show_groups=False
market_map.selected = ['PAN', 'FRA', 'PHL']
# changing selected stroke and hovered stroke variable
market_map.selected_stroke = 'yellow'
market_map.hovered_stroke = 'violet'

In [ ]:
# Adding data for color and making color axis visible
market_map.color = data['GDP']
ax_c.visible = True

# Graphs

In [ ]:
# Creating the figure to be displayed as the tooltip
sc_x = DateScale()
sc_y = LinearScale()

ax_x = Axis(scale=sc_x, grid_lines='dashed', label='Date')
ax_y = Axis(scale=sc_y, orientation='vertical', grid_lines='dashed',
            label='GDP', label_location='end', label_offset='-1em')

line = Lines(x= gdp_data.index.values, scales={'x': sc_x, 'y': sc_y}, colors=['orange'])
fig_tooltip = Figure(marks=[line], axes=[ax_x, ax_y], min_width=600, min_height=400)

In [ ]:
market_map = MarketMap(names=country_codes, groups=continents,
                       cols=25, row_groups=3,
                       color=data['GDP'], scales={'color': col}, axes=[ax_c],
                       ref_data=data, tooltip_widget=fig_tooltip)

# Update the tooltip chart
hovered_symbol = ''
def hover_handler(self, content):
    global hovered_symbol
    symbol = content.get('ref_data', {}).get('Country Code', '')
    if(symbol != hovered_symbol):
        hovered_symbol = symbol
        if(gdp_data.get(hovered_symbol) is not None):
            line.y = gdp_data[hovered_symbol].values
            fig_tooltip.title = hovered_symbol
               
# Custom msg sent when a particular cell is hovered on
market_map.on_hover(hover_handler)
market_map.axes = []
display(market_map)

# https://github.com/bloomberg/bqplot